In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import re #dùng để tìm kiếm, kiểm tra, thay thế và xử lý chuỗi (string) theo mẫu (pattern)

%matplotlib inline
pd.set_option('display.max_rows', None) # Hiển thị biểu đồ matplotlib trực tiếp trong notebook (ngay dưới cell code)
pd.set_option('display.max_columns', None) # Cho phép hiển thị toàn bộ số dòng của DataFrame khi print(df) hoặc df

# Data Cleaning và Preprocessing cho Amazon Sale Dataset

## 1. Tải dữ liệu (Load Data)


In [33]:
# Tải dữ liệu
df = pd.read_csv('Amazon_SaleData.csv')

# Hiển thị thông tin cơ bản về dataset
print("Shape của dataset:", df.shape)
print("\nCác cột trong dataset:")
print(df.columns.tolist())
print("\n5 dòng đầu tiên:")
df.head()

Shape của dataset: (100000, 20)

Các cột trong dataset:
['OrderID', 'OrderDate', 'CustomerID', 'CustomerName', 'ProductID', 'ProductName', 'Category', 'Brand', 'Quantity', 'UnitPrice', 'Discount', 'Tax', 'ShippingCost', 'TotalAmount', 'PaymentMethod', 'OrderStatus', 'City', 'State', 'Country', 'SellerID']

5 dòng đầu tiên:


,OrderID,OrderDate,CustomerID,CustomerName,ProductID,ProductName,Category,Brand,Quantity,UnitPrice,Discount,Tax,ShippingCost,TotalAmount,PaymentMethod,OrderStatus,City,State,Country,SellerID
0,ORD0000001,2023-01-31,CUST001504,Vihaan Sharma,P00014,Drone Mini,Books,BrightLux,3,106.59,0.00,0.00,0.09,319.86,Debit Card,Delivered,Washington,DC,India,SELL01967
1,ORD0000002,2023-12-30,CUST000178,Pooja Kumar,P00040,Microphone,Home & Kitchen,UrbanStyle,1,251.37,0.05,19.10,1.74,259.64,Amazon Pay,Delivered,Fort Worth,TX,United States,SELL01298
2,ORD0000003,2022-05-10,CUST047516,Sneha Singh,P00044,Power Bank 20000mAh,Clothing,UrbanStyle,3,35.03,0.10,7.57,5.91,108.06,Debit Card,Delivered,Austin,TX,United States,SELL00908
3,ORD0000004,2023-07-18,CUST030059,Vihaan Reddy,P00041,Webcam Full HD,Home & Kitchen,Zenith,5,33.58,0.15,11.42,5.53,159.66,Cash on Delivery,Delivered,Charlotte,NC,India,SELL01164
4,ORD0000005,2023-02-04,CUST048677,Aditya Kapoor,P00029,T-Shirt,Clothing,KiddoFun,2,515.64,0.25,38.67,9.23,821.36,Credit Card,Cancelled,San Antonio,TX,Canada,SELL01411


## 2. Khám phá dữ liệu ban đầu (Initial Data Exploration)

kiểu dữ liệu, thống kê mô tả, và xác định các vấn đề tiềm ẩn.

In [34]:
# Kiểu dữ liệu của các cột
print("Kiểu dữ liệu (Data Types):")
print(df.dtypes)
print("\n" + "="*50)

# Thông tin tổng quan
print("\nThông tin tổng quan về dataset:")
df.info()
print("\n" + "="*50)

# Thống kê mô tả cho các cột số
print("\nThống kê mô tả cho các cột số:")
df.describe()

Kiểu dữ liệu (Data Types):
OrderID           object
OrderDate         object
CustomerID        object
CustomerName      object
ProductID         object
ProductName       object
Category          object
Brand             object
Quantity           int64
UnitPrice        float64
Discount         float64
Tax              float64
ShippingCost     float64
TotalAmount      float64
PaymentMethod     object
OrderStatus       object
City              object
State             object
Country           object
SellerID          object
dtype: object


Thông tin tổng quan về dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   OrderID        100000 non-null  object 
 1   OrderDate      100000 non-null  object 
 2   CustomerID     100000 non-null  object 
 3   CustomerName   100000 non-null  object 
 4   ProductID      100000 non-null  object 


,Quantity,UnitPrice,Discount,Tax,ShippingCost,TotalAmount
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,3.001400,302.905748,0.074226,68.468902,7.406660,918.256479
std,1.413548,171.840797,0.082583,74.131180,4.324057,724.508332
min,1.000000,5.000000,0.000000,0.000000,0.000000,4.270000
25%,2.000000,154.190000,0.000000,15.920000,3.680000,340.890000
50%,3.000000,303.070000,0.050000,45.250000,7.300000,714.315000
75%,4.000000,451.500000,0.100000,96.060000,11.150000,1349.765000
max,5.000000,599.990000,0.300000,538.460000,15.000000,3534.980000


## 3. Kiểm tra và xử lý dữ liệu trùng lặp (Check and Handle Duplicates)

Xác định và loại bỏ các bản ghi trùng lặp trong dataset để đảm bảo chất lượng dữ liệu.

In [35]:
# Kiểm tra số lượng bản ghi trùng lặp
duplicate_count = df.duplicated().sum()
print(f"Số lượng bản ghi trùng lặp: {duplicate_count}")

# Loại bỏ các bản ghi trùng lặp nếu có
if duplicate_count > 0:
    df = df.drop_duplicates()
    print(f"Đã xóa {duplicate_count} bản ghi trùng lặp")
    print(f"Shape sau khi xóa duplicates: {df.shape}")
else:
    print("Không có bản ghi trùng lặp trong dataset")

Số lượng bản ghi trùng lặp: 0
Không có bản ghi trùng lặp trong dataset


## 4. Kiểm tra và xử lý giá trị thiếu (Handle Missing Values)

Kiểm tra các giá trị thiếu (missing values) trong dataset và quyết định phương pháp xử lý phù hợp (điền khuyết hoặc xóa).

In [36]:
# Kiểm tra giá trị thiếu
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Percentage': missing_percentage
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing_df) > 0:
    print("Các cột có giá trị thiếu:")
    print(missing_df)
    print("\n" + "="*50)
else:
    print("Không có giá trị thiếu trong dataset")
    print("\n" + "="*50)

# Hiển thị tổng quan về missing values
print(f"\nTổng số giá trị thiếu: {df.isnull().sum().sum()}")
print(f"Tỷ lệ giá trị thiếu: {(df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100:.2f}%")

Không có giá trị thiếu trong dataset




Tổng số giá trị thiếu: 0
Tỷ lệ giá trị thiếu: 0.00%


### 4.1. Điền khuyết cho các cột số (Fill Missing Values for Numeric Columns)

Sử dụng phương pháp thích hợp để điền giá trị thiếu cho các cột số (trung bình, median, hoặc mode).

In [37]:
# Lưu bản sao trước khi xử lý
df_backup = df.copy()

# Xác định các cột số
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
print("Các cột số trong dataset:", numeric_columns)

# Điền giá trị thiếu cho cột số bằng median (ít bị ảnh hưởng bởi outliers hơn mean)
for col in numeric_columns:
    if df[col].isnull().sum() > 0:
        median_value = df[col].median()
        df[col].fillna(median_value, inplace=True)
        print(f"Đã điền {df[col].isnull().sum()} giá trị thiếu trong cột '{col}' bằng median: {median_value}")

Các cột số trong dataset: ['Quantity', 'UnitPrice', 'Discount', 'Tax', 'ShippingCost', 'TotalAmount']


### 4.2. Điền khuyết cho các cột phân loại (Fill Missing Values for Categorical Columns)

Xử lý giá trị thiếu cho các cột phân loại bằng cách điền mode (giá trị xuất hiện nhiều nhất) hoặc giá trị mặc định.

In [38]:
# Xác định các cột phân loại (object/string)
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
print("Các cột phân loại trong dataset:", categorical_columns)

# Điền giá trị thiếu cho cột phân loại bằng mode (giá trị xuất hiện nhiều nhất)
for col in categorical_columns:
    if df[col].isnull().sum() > 0:
        mode_value = df[col].mode()[0] if len(df[col].mode()) > 0 else 'Unknown'
        df[col].fillna(mode_value, inplace=True)
        print(f"Đã điền {df[col].isnull().sum()} giá trị thiếu trong cột '{col}' bằng mode: {mode_value}")

# Kiểm tra lại sau khi điền khuyết
print(f"\nTổng số giá trị thiếu sau khi điền khuyết: {df.isnull().sum().sum()}")

Các cột phân loại trong dataset: ['OrderID', 'OrderDate', 'CustomerID', 'CustomerName', 'ProductID', 'ProductName', 'Category', 'Brand', 'PaymentMethod', 'OrderStatus', 'City', 'State', 'Country', 'SellerID']

Tổng số giá trị thiếu sau khi điền khuyết: 0


## 5. Chuẩn hóa kiểu dữ liệu (Data Type Standardization)

Chuyển đổi các cột sang kiểu dữ liệu phù hợp để tối ưu hóa việc xử lý và phân tích (ví dụ: OrderDate sang datetime).

In [39]:
# Chuyển đổi OrderDate sang datetime
print("Kiểu dữ liệu của OrderDate trước khi chuyển đổi:", df['OrderDate'].dtype)
df['OrderDate'] = pd.to_datetime(df['OrderDate'], errors='coerce')

# Kiểm tra các giá trị không hợp lệ sau khi chuyển đổi
invalid_dates = df['OrderDate'].isnull().sum()
if invalid_dates > 0:
    print(f"Cảnh báo: Có {invalid_dates} giá trị ngày không hợp lệ")
    # Xóa các dòng có ngày không hợp lệ nếu cần
    # df = df.dropna(subset=['OrderDate'])
else:
    print("Tất cả giá trị ngày đều hợp lệ")

print("\nKiểu dữ liệu của OrderDate sau khi chuyển đổi:", df['OrderDate'].dtype)
print("\nPhạm vi ngày trong dataset:")
print(f"Ngày sớm nhất: {df['OrderDate'].min()}")
print(f"Ngày muộn nhất: {df['OrderDate'].max()}")

Kiểu dữ liệu của OrderDate trước khi chuyển đổi: object
Tất cả giá trị ngày đều hợp lệ

Kiểu dữ liệu của OrderDate sau khi chuyển đổi: datetime64[ns]

Phạm vi ngày trong dataset:
Ngày sớm nhất: 2020-01-01 00:00:00
Ngày muộn nhất: 2024-12-29 00:00:00


## 6. Làm sạch cột văn bản (Text Column Cleaning)

Loại bỏ khoảng trắng thừa, chuẩn hóa định dạng, và xử lý các ký tự đặc biệt trong các cột văn bản.

In [40]:
# Làm sạch các cột văn bản: loại bỏ khoảng trắng đầu/cuối
text_columns = ['CustomerName', 'ProductName', 'Category', 'Brand', 'City', 'State', 'Country', 
                'PaymentMethod', 'OrderStatus']

for col in text_columns:
    if col in df.columns:
        # Loại bỏ khoảng trắng đầu và cuối
        df[col] = df[col].astype(str).str.strip()
        # Loại bỏ khoảng trắng thừa ở giữa
        df[col] = df[col].str.replace(r'\s+', ' ', regex=True)
        print(f"Đã làm sạch cột '{col}'")

print("\nĐã hoàn thành việc làm sạch các cột văn bản")

Đã làm sạch cột 'CustomerName'
Đã làm sạch cột 'ProductName'
Đã làm sạch cột 'Category'
Đã làm sạch cột 'Brand'
Đã làm sạch cột 'City'
Đã làm sạch cột 'State'
Đã làm sạch cột 'Country'
Đã làm sạch cột 'PaymentMethod'
Đã làm sạch cột 'OrderStatus'

Đã hoàn thành việc làm sạch các cột văn bản


## 7. Chuẩn hóa dữ liệu phân loại (Categorical Data Standardization)

Chuẩn hóa các giá trị trong cột phân loại để đảm bảo tính nhất quán (ví dụ: viết hoa/chữ thường, loại bỏ biến thể).

In [41]:
# Chuẩn hóa các giá trị phân loại quan trọng
# Chuẩn hóa OrderStatus
print("Các giá trị OrderStatus trước khi chuẩn hóa:")
print(df['OrderStatus'].value_counts())

df['OrderStatus'] = df['OrderStatus'].str.title()
print("\nCác giá trị OrderStatus sau khi chuẩn hóa:")
print(df['OrderStatus'].value_counts())
print("\n" + "="*50)

# Chuẩn hóa PaymentMethod
print("\nCác giá trị PaymentMethod trước khi chuẩn hóa:")
print(df['PaymentMethod'].value_counts())

df['PaymentMethod'] = df['PaymentMethod'].str.title()
print("\nCác giá trị PaymentMethod sau khi chuẩn hóa:")
print(df['PaymentMethod'].value_counts())

Các giá trị OrderStatus trước khi chuẩn hóa:
OrderStatus
Delivered    74628
Shipped      15192
Pending       4103
Returned      3049
Cancelled     3028
Name: count, dtype: int64

Các giá trị OrderStatus sau khi chuẩn hóa:
OrderStatus
Delivered    74628
Shipped      15192
Pending       4103
Returned      3049
Cancelled     3028
Name: count, dtype: int64


Các giá trị PaymentMethod trước khi chuẩn hóa:
PaymentMethod
Credit Card         35038
Debit Card          20024
UPI                 15066
Amazon Pay          15017
Net Banking          9927
Cash on Delivery     4928
Name: count, dtype: int64

Các giá trị PaymentMethod sau khi chuẩn hóa:
PaymentMethod
Credit Card         35038
Debit Card          20024
Upi                 15066
Amazon Pay          15017
Net Banking          9927
Cash On Delivery     4928
Name: count, dtype: int64


## 8. Xử lý ngoại lai (Outlier Detection and Treatment)

Phát hiện và xử lý các giá trị ngoại lai (outliers) trong các cột số có thể ảnh hưởng đến phân tích.

In [42]:
# Xác định outliers sử dụng IQR method cho các cột số quan trọng
numeric_cols_for_outlier = ['Quantity', 'UnitPrice', 'Discount', 'Tax', 'ShippingCost', 'TotalAmount']

def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

print("Phát hiện outliers sử dụng IQR method:")
print("="*60)
for col in numeric_cols_for_outlier:
    outliers, lower, upper = detect_outliers_iqr(df, col)
    outlier_count = len(outliers)
    percentage = (outlier_count / len(df)) * 100
    print(f"\n{col}:")
    print(f"  - Lower bound: {lower:.2f}")
    print(f"  - Upper bound: {upper:.2f}")
    print(f"  - Số lượng outliers: {outlier_count} ({percentage:.2f}%)")

Phát hiện outliers sử dụng IQR method:

Quantity:
  - Lower bound: -1.00
  - Upper bound: 7.00
  - Số lượng outliers: 0 (0.00%)

UnitPrice:
  - Lower bound: -291.78
  - Upper bound: 897.47
  - Số lượng outliers: 0 (0.00%)

Discount:
  - Lower bound: -0.15
  - Upper bound: 0.25
  - Số lượng outliers: 1989 (1.99%)

Tax:
  - Lower bound: -104.29
  - Upper bound: 216.27
  - Số lượng outliers: 5109 (5.11%)

ShippingCost:
  - Lower bound: -7.53
  - Upper bound: 22.36
  - Số lượng outliers: 0 (0.00%)

TotalAmount:
  - Lower bound: -1172.42
  - Upper bound: 2863.08
  - Số lượng outliers: 1360 (1.36%)


### 8.1. Xử lý outliers (Outlier Treatment)

Có thể capping/winsorizing outliers hoặc xóa chúng tùy thuộc vào yêu cầu phân tích. Ở đây, chúng ta sẽ sử dụng phương pháp capping (giới hạn giá trị).

In [43]:
# Xử lý outliers bằng capping (giới hạn giá trị ở ngưỡng IQR)
# Lưu ý: Có thể bỏ qua bước này nếu outliers là hợp lệ trong ngữ cảnh nghiệp vụ

def cap_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Đảm bảo không capping dưới giá trị tối thiểu hợp lý (ví dụ: không âm cho giá, số lượng)
    if column in ['Quantity', 'UnitPrice', 'Discount', 'Tax', 'ShippingCost', 'TotalAmount']:
        lower_bound = max(0, lower_bound)
    
    data[column] = data[column].clip(lower=lower_bound, upper=upper_bound)
    return data

# Áp dụng capping cho các cột cụ thể (có thể bỏ qua nếu cần giữ nguyên outliers)
for col in ['UnitPrice', 'Discount', 'Tax', 'ShippingCost', 'TotalAmount']:
    df = cap_outliers_iqr(df, col)
    print(f"Đã áp dụng capping cho cột {col}")


Đã áp dụng capping cho cột UnitPrice
Đã áp dụng capping cho cột Discount
Đã áp dụng capping cho cột Tax
Đã áp dụng capping cho cột ShippingCost
Đã áp dụng capping cho cột TotalAmount


## 9. Kiểm tra tính nhất quán dữ liệu (Data Consistency Check)

Kiểm tra tính nhất quán logic của dữ liệu, ví dụ: TotalAmount có bằng (UnitPrice * Quantity * (1-Discount)) + Tax + ShippingCost không?

In [44]:
# Tính toán TotalAmount theo công thức
df['CalculatedTotal'] = (df['UnitPrice'] * df['Quantity'] * (1 - df['Discount'])) + df['Tax'] + df['ShippingCost']

# So sánh với TotalAmount hiện tại (cho phép sai số nhỏ do làm tròn)
df['AmountDifference'] = abs(df['TotalAmount'] - df['CalculatedTotal'])
tolerance = 0.01  # Cho phép sai số 0.01

inconsistent_rows = df[df['AmountDifference'] > tolerance]
print(f"Số dòng có TotalAmount không khớp với công thức: {len(inconsistent_rows)}")
print(f"Tỷ lệ: {len(inconsistent_rows)/len(df)*100:.2f}%")

if len(inconsistent_rows) > 0:
    print("\nMột số ví dụ về dữ liệu không nhất quán:")
    print(inconsistent_rows[['OrderID', 'UnitPrice', 'Quantity', 'Discount', 'Tax', 'ShippingCost', 
                             'TotalAmount', 'CalculatedTotal', 'AmountDifference']].head(10))
    
    # Có thể cập nhật TotalAmount bằng giá trị tính toán
    # Uncomment dòng dưới nếu muốn sửa
    # df['TotalAmount'] = df['CalculatedTotal']
else:
    print("Tất cả dữ liệu đều nhất quán!")

# Xóa các cột tạm thời
df = df.drop(['CalculatedTotal', 'AmountDifference'], axis=1, errors='ignore')

Số dòng có TotalAmount không khớp với công thức: 7481
Tỷ lệ: 7.48%

Một số ví dụ về dữ liệu không nhất quán:
        OrderID  UnitPrice  Quantity  Discount     Tax  ShippingCost  \
18   ORD0000019     529.19         5      0.00  216.27          4.57   
52   ORD0000053     324.76         3      0.25   34.10          0.75   
58   ORD0000059     184.70         3      0.25   46.54          7.98   
60   ORD0000061     593.02         4      0.05  216.27          9.18   
72   ORD0000073     582.66         5      0.00  216.27         14.07   
75   ORD0000076     437.47         4      0.00  216.27          8.82   
84   ORD0000085     494.13         4      0.05  216.27          7.18   
91   ORD0000092     555.99         4      0.05  216.27          1.26   
93   ORD0000094     409.78         4      0.00  216.27         14.29   
111  ORD0000112     314.00         5      0.00  216.27          8.14   

     TotalAmount  CalculatedTotal  AmountDifference  
18     2863.0775         2866.790           

## 10. Kiểm tra giá trị không hợp lệ (Invalid Value Check)

Kiểm tra và xử lý các giá trị không hợp lệ (ví dụ: số lượng âm, giá âm, discount ngoài phạm vi 0-1, v.v.).

In [45]:
# Kiểm tra các giá trị không hợp lệ
print("Kiểm tra giá trị không hợp lệ:")
print("="*50)

# Kiểm tra Quantity (phải là số nguyên dương)
invalid_quantity = df[df['Quantity'] <= 0]
print(f"Số dòng có Quantity <= 0: {len(invalid_quantity)}")
if len(invalid_quantity) > 0:
    df = df[df['Quantity'] > 0]  # Xóa các dòng không hợp lệ
    print(f"Đã xóa {len(invalid_quantity)} dòng có Quantity không hợp lệ")

# Kiểm tra UnitPrice (phải >= 0)
invalid_price = df[df['UnitPrice'] < 0]
print(f"\nSố dòng có UnitPrice < 0: {len(invalid_price)}")
if len(invalid_price) > 0:
    df = df[df['UnitPrice'] >= 0]
    print(f"Đã xóa {len(invalid_price)} dòng có UnitPrice không hợp lệ")

# Kiểm tra Discount (thường nằm trong khoảng 0-1 hoặc 0-100)
if df['Discount'].max() > 1:
    # Nếu discount > 1, có thể là phần trăm, chuẩn hóa về 0-1
    df['Discount'] = df['Discount'] / 100
    print("\nĐã chuẩn hóa Discount từ phần trăm sang tỷ lệ (0-1)")

invalid_discount = df[(df['Discount'] < 0) | (df['Discount'] > 1)]
print(f"\nSố dòng có Discount ngoài phạm vi [0,1]: {len(invalid_discount)}")
if len(invalid_discount) > 0:
    df = df[(df['Discount'] >= 0) & (df['Discount'] <= 1)]
    print(f"Đã xóa {len(invalid_discount)} dòng có Discount không hợp lệ")

# Kiểm tra Tax và ShippingCost (phải >= 0)
invalid_tax = df[df['Tax'] < 0]
invalid_shipping = df[df['ShippingCost'] < 0]
print(f"\nSố dòng có Tax < 0: {len(invalid_tax)}")
print(f"Số dòng có ShippingCost < 0: {len(invalid_shipping)}")
if len(invalid_tax) > 0:
    df = df[df['Tax'] >= 0]
if len(invalid_shipping) > 0:
    df = df[df['ShippingCost'] >= 0]

print(f"\nShape sau khi xử lý giá trị không hợp lệ: {df.shape}")

Kiểm tra giá trị không hợp lệ:
Số dòng có Quantity <= 0: 0

Số dòng có UnitPrice < 0: 0

Số dòng có Discount ngoài phạm vi [0,1]: 0

Số dòng có Tax < 0: 0
Số dòng có ShippingCost < 0: 0

Shape sau khi xử lý giá trị không hợp lệ: (100000, 20)


## 11. Feature Engineering: 



In [46]:
# Tạo thêm các cột ngày trong tuần
df["day_of_week"] = df["OrderDate"].dt.dayofweek

# Tạo thêm cột revenue (Doanh thu thực tế chỉ có khi mà order đã hoàn thành (thành công))
df["Revenue"] = df.apply(lambda row: row["TotalAmount"] if row["OrderStatus"] == "Delivered" else 0, axis=1)

## 12. Tổng kết và lưu dữ liệu đã làm sạch (Summary and Save Cleaned Data)

Tổng kết các thay đổi đã thực hiện và lưu dataset đã được làm sạch.

In [47]:
# Tổng kết
print("="*60)
print("TỔNG KẾT QUÁ TRÌNH LÀM SẠCH DỮ LIỆU")
print("="*60)
print(f"\nShape ban đầu: {df_backup.shape}")
print(f"Shape sau khi làm sạch: {df.shape}")
print(f"Số dòng đã xóa: {len(df_backup) - len(df)}")

print("\nThông tin dataset sau khi làm sạch:")
print(f"- Số cột: {df.shape[1]}")
print(f"- Số dòng: {df.shape[0]}")
print(f"- Giá trị thiếu: {df.isnull().sum().sum()}")
print(f"- Dữ liệu trùng lặp: {df.duplicated().sum()}")

print("\nKiểu dữ liệu cuối cùng:")
print(df.dtypes)

print("\n5 dòng đầu tiên của dataset đã làm sạch:")
df.head()

TỔNG KẾT QUÁ TRÌNH LÀM SẠCH DỮ LIỆU

Shape ban đầu: (100000, 20)
Shape sau khi làm sạch: (100000, 22)
Số dòng đã xóa: 0

Thông tin dataset sau khi làm sạch:
- Số cột: 22
- Số dòng: 100000
- Giá trị thiếu: 0
- Dữ liệu trùng lặp: 0

Kiểu dữ liệu cuối cùng:
OrderID                  object
OrderDate        datetime64[ns]
CustomerID               object
CustomerName             object
ProductID                object
ProductName              object
Category                 object
Brand                    object
Quantity                  int64
UnitPrice               float64
Discount                float64
Tax                     float64
ShippingCost            float64
TotalAmount             float64
PaymentMethod            object
OrderStatus              object
City                     object
State                    object
Country                  object
SellerID                 object
day_of_week               int32
Revenue                 float64
dtype: object

5 dòng đầu tiên của datase

,OrderID,OrderDate,CustomerID,CustomerName,ProductID,ProductName,Category,Brand,Quantity,UnitPrice,Discount,Tax,ShippingCost,TotalAmount,PaymentMethod,OrderStatus,City,State,Country,SellerID,day_of_week,Revenue
0,ORD0000001,2023-01-31,CUST001504,Vihaan Sharma,P00014,Drone Mini,Books,BrightLux,3,106.59,0.00,0.00,0.09,319.86,Debit Card,Delivered,Washington,DC,India,SELL01967,1,319.86
1,ORD0000002,2023-12-30,CUST000178,Pooja Kumar,P00040,Microphone,Home & Kitchen,UrbanStyle,1,251.37,0.05,19.10,1.74,259.64,Amazon Pay,Delivered,Fort Worth,TX,United States,SELL01298,5,259.64
2,ORD0000003,2022-05-10,CUST047516,Sneha Singh,P00044,Power Bank 20000mAh,Clothing,UrbanStyle,3,35.03,0.10,7.57,5.91,108.06,Debit Card,Delivered,Austin,TX,United States,SELL00908,1,108.06
3,ORD0000004,2023-07-18,CUST030059,Vihaan Reddy,P00041,Webcam Full HD,Home & Kitchen,Zenith,5,33.58,0.15,11.42,5.53,159.66,Cash On Delivery,Delivered,Charlotte,NC,India,SELL01164,1,159.66
4,ORD0000005,2023-02-04,CUST048677,Aditya Kapoor,P00029,T-Shirt,Clothing,KiddoFun,2,515.64,0.25,38.67,9.23,821.36,Credit Card,Cancelled,San Antonio,TX,Canada,SELL01411,5,0.00


Lưu dataset đã được làm sạch vào file CSV mới để sử dụng cho các bước phân tích tiếp theo.

In [48]:
# Lưu dữ liệu đã làm sạch
output_filename = 'Amazon_SaleData_Cleaned.csv'
df.to_csv(output_filename, index=False)
print(f"Đã lưu dataset đã làm sạch vào file: {output_filename}")
print(f"File đã được lưu tại: {output_filename}")

# Xác nhận bằng cách đọc lại file
df_verify = pd.read_csv(output_filename)
print(f"\nXác nhận: Đã đọc lại file thành công. Shape: {df_verify.shape}")

Đã lưu dataset đã làm sạch vào file: Amazon_SaleData_Cleaned.csv
File đã được lưu tại: Amazon_SaleData_Cleaned.csv

Xác nhận: Đã đọc lại file thành công. Shape: (100000, 22)
